In [15]:
import numpy as np
import pymc as pm
import scipy.stats as stats
from IPython.core.pylabtools import figsize

figsize(12.5, 4)

data = np.loadtxt("mixture_data.csv", delimiter=",")

# p for the probability of belonging to cluster 0, ~p cluster 1
p = pm.Uniform("p", 0, 1)

# assignment of the clusters randomly
assignment = pm.Categorical("assignment", [p, 1 -p], size=data.shape[0])

# the variance of each cluster - to the power 2!
taus = 1.0/pm.Uniform("stds", 0, 100, size=2) ** 2
# the mean of each cluster > this we get from the data, so maybe it is 
# emperical Bayes instead of Bayes?
centers = pm.Normal("centers", [120, 190], [.01, .01], size=2)

@pm.deterministic
def center_i(assignment=assignment, centers=centers):
    return centers[assignment]

@pm.deterministic
def tau_i(assignment=assignment, taus=taus): 
    return taus[assignment]


In [23]:
observation = pm.Normal("obs", center_i, tau_i, value = data, )

modle = pm.Model([p, assignment, observation, taus, centers])

In [21]:
taus.value[0:10]


array([ 0.00029522,  0.00011543])